# UBS_lauzhack - Entity resolution model

## 1. Pre-processing

Observed the given dataset and apply the proper transformations to the data in order to better anlyse it and create the model.

In [1]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
import warnings


InteractiveShell.ast_node_interactivity = "all"
#pd.set_option('precision', 3)
display.precision = 3
sns.set()
warnings.filterwarnings('ignore')

# extra imports
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
#from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox



In [2]:
# Read the CSV files from the 'data' subfolder
account_booking_df = pd.read_csv('data/account_booking_train.csv')
external_parties_df = pd.read_csv('data/external_parties_train.csv')

In [3]:
# Combine the tables with merge using 
combined_data = pd.merge(account_booking_df, external_parties_df, on='transaction_reference_id', how='inner')

In [4]:
#Null count of each variables to know which ones are more relevant to analyze
combined_data.isnull().sum()

transaction_reference_id            0
debit_credit_indicator              0
account_id                          0
transaction_amount                  0
transaction_currency                0
transaction_date                    0
party_role                          0
party_info_unstructured             0
parsed_name                         0
parsed_address_street_name        739
parsed_address_street_number     3030
parsed_address_unit             11064
parsed_address_postal_code       3480
parsed_address_city              1168
parsed_address_state             9107
parsed_address_country           6490
party_iban                       3399
party_phone                      5000
external_id                         0
dtype: int64

In [5]:
# Drop the columns that have more than 50% of missing values and irrelevant information

## External parties data

irrelevant_cols_external = ['party_info_unstructured', 'parsed_address_unit', 'parsed_address_state', 'parsed_address_country']
external_parties_df.drop(columns=irrelevant_cols_external, inplace=True, errors='ignore')

## Accounts booking data
duplicate_ids = account_booking_df[account_booking_df.duplicated(subset='transaction_reference_id', keep=False)]
account_booking_df = account_booking_df[~account_booking_df['transaction_reference_id'].isin(duplicate_ids['transaction_reference_id'])]

irrelevant_cols_booking = ['debit_credit_indicator']
account_booking_df.drop(columns=irrelevant_cols_booking, inplace=True, errors='ignore')

# Merge the two dataframes

merged_df = pd.merge(external_parties_df, account_booking_df, on='transaction_reference_id', how='inner')

merged_df.to_csv('merged_data_cleaned.csv', index=False)

In [6]:

honorifics = ['Mr. ', 'Ms. ', 'Mrs. ', 'Miss ', 'Dr. ', 'Prof. ', 'Rev. ', 'Hon. ', 'mr. ', 'ms. ', 'mrs. ', 'miss ', 'dr. ', 'prof. ', 'rev. ', 'hon. ']
for honorific in honorifics:
    merged_df['parsed_name'] = merged_df['parsed_name'].str.replace(honorific, '')


def delete_duplicates(name):
    def remove_duplicates(text):
        words = text.split()
        seen = set()
        result = []
        for word in words:
            if word.lower() not in seen:
                seen.add(word.lower())
                result.append(word)
        return ' '.join(result)
    
    name['parsed_name'] = name['parsed_name'].apply(remove_duplicates)
    return name

merged_df = delete_duplicates(merged_df)


In [7]:
from metaphone import doublemetaphone

def apply_metaphone(value):
    if isinstance(value, list) or isinstance(value, np.ndarray):  
        return np.nan  
    if pd.isna(value) or not isinstance(value, str):  
        return np.nan
    return doublemetaphone(value.strip().lower())[0] 

def split_and_metaphone(full_name):
    """
    Divide un nombre completo en sus partes (palabras), aplica Metaphone a cada parte,
    y las junta nuevamente con espacios.
    """
    if pd.isna(full_name) or not isinstance(full_name, str):  
        return np.nan

    parts = full_name.strip().split()

    return ' '.join(apply_metaphone(part) for part in parts)

merged_df['parsed_name'] = merged_df['parsed_name'].apply(split_and_metaphone)

print(merged_df['parsed_name'].head())


0          MR M0
1         ASN KM
2      RPRSN JRR
3    ASKSSNLSN K
4      MMJJ PT A
Name: parsed_name, dtype: object


In [8]:
merged_df['parsed_address_street_name'] = merged_df['parsed_address_street_name'].apply(split_and_metaphone)

print(merged_df['parsed_address_street_name'].head())

merged_df.to_csv('merged_data_cleaned.csv', index=False)

0       ANJL PRKS
1         NR0 MKL
2    RXL KRSNKPTT
3        PRTS ST 
4    JN LK SS KLN
Name: parsed_address_street_name, dtype: object


In [5]:
import jellyfish  # Para similitudes como Jaro-Winkler
from fuzzywuzzy import fuzz  # Para similitudes adicionales, si necesario

# Función para calcular similitudes entre dos nombres
def name_similarity(name1, name2):
    # Utilizamos Jaro-Winkler para calcular la similitud de los nombres
    return jellyfish.jaro_winkler(name1, name2)

# Función para calcular similitud entre dos números de teléfono
def phone_similarity(phone1, phone2):
    return phone1 == phone2  # Comparación exacta después del preprocesamiento

# Función para comparar códigos postales
def postal_code_similarity(code1, code2):
    return code1 == code2  # Comparación exacta

# Crear función para calcular similitudes generales entre los pares
def calculate_similarities(df, pairs):
    similarities = []
    
    for pair in pairs:
        idx1, idx2 = pair
        row1, row2 = df.loc[idx1], df.loc[idx2]
        
        # Calcular la similitud de nombres
        name_sim = name_similarity(row1['parsed_name'], row2['parsed_name'])
        
        # Calcular la similitud de teléfonos
        phone_sim = phone_similarity(row1['party_phone'], row2['party_phone'])
        
        # Calcular la similitud de códigos postales
        postal_sim = postal_code_similarity(row1['postal_code'], row2['postal_code'])
        
        # Podríamos ponderar las similitudes si es necesario (por ejemplo, más peso al nombre)
        total_similarity = (name_sim * 0.7) + (phone_sim * 0.2) + (postal_sim * 0.1)
        
        similarities.append(total_similarity)
    
    return similarities

# Calcular las similitudes para los pares generados
similarities = calculate_similaristies(merged_df, pairs)

# Mostrar algunas de las similitudes calculadas
similarities[:10]  # Primeros 10 resultados


NameError: name 'merged_df' is not defined

In [16]:
from itertools import combinations

# Generar pares de transacciones dentro de cada bloque
def create_pairs(df, block_column):
    pairs = []
    for _, block in df.groupby(block_column):
        block_pairs = list(combinations(block.index, 2))
        pairs.extend(block_pairs)
    return pairs

# Crear los pares usando el 'block_key' fonético
pairs = create_pairs(merged_df, 'block_key')

KeyError: 'block_key'

### 1.1 Pre-processing of the variable parsed_name

In [9]:
# Delete honorifics from the names

# Sample data for demonstration
merged_data_cleaned2 = pd.DataFrame({
    'transaction_reference_id': [1, 2, 3, 4, 5],
    'parsed_name': ['Mr. John Doe', 'Dr. Jon Doe', 'Ms. Jane Doe', 'Jonny Doe', 'Prof. Sonya Sonya'],
    'postal_code': ['12345', '12345', '67890', '67891', '67892']
})

def delete_honorifics(name): 
    name['parsed_name'] = name['parsed_name'].str.replace('Mr. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Ms. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Mrs. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Miss ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Dr. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Prof. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Rev. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Hon. ', '')

delete_honorifics(merged_data_cleaned2)
print(merged_data_cleaned2)

   transaction_reference_id  parsed_name postal_code
0                         1     John Doe       12345
1                         2      Jon Doe       12345
2                         3     Jane Doe       67890
3                         4    Jonny Doe       67891
4                         5  Sonya Sonya       67892


In [10]:
# Normalize the names

def normalize_names(name):
    name['parsed_name'] = name['parsed_name'].str.lower()

normalize_names(merged_data_cleaned2)
print(merged_data_cleaned2)

   transaction_reference_id  parsed_name postal_code
0                         1     john doe       12345
1                         2      jon doe       12345
2                         3     jane doe       67890
3                         4    jonny doe       67891
4                         5  sonya sonya       67892


In [11]:
# Delete duplicates in the names

# Function to delete duplicated words in names
def delete_duplicates(name):
    def remove_duplicates(text):
        words = text.split()
        seen = set()
        result = []
        for word in words:
            if word.lower() not in seen:
                seen.add(word.lower())
                result.append(word)
        return ' '.join(result)
    
    name['parsed_name'] = name['parsed_name'].apply(remove_duplicates)
    return name

delete_duplicates(merged_data_cleaned2) 
print(merged_data_cleaned2)

,transaction_reference_id,parsed_name,postal_code
0,1,john doe,12345
1,2,jon doe,12345
2,3,jane doe,67890
3,4,jonny doe,67891
4,5,sonya,67892


   transaction_reference_id parsed_name postal_code
0                         1    john doe       12345
1                         2     jon doe       12345
2                         3    jane doe       67890
3                         4   jonny doe       67891
4                         5       sonya       67892


In [12]:
print(merged_data_cleaned2.columns)


Index(['transaction_reference_id', 'parsed_name', 'postal_code'], dtype='object')


In [13]:
import networkx as nx
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
#from jellyfish import jaro_winkler_similarity
import pandas as pd
import fuzzy

# Initialize Soundex or Metaphone
metaphone = fuzzy.nysiis  # Uncomment this line to use Metaphone

# Step 2: Indexing (blocking) using Soundex or Metaphone
def create_block_key(name):
    metaphone_code = metaphone(name)  
    return metaphone_code

# Create block keys
#merged_data_cleaned2['block_key'] = merged_data_cleaned2.apply(lambda row: create_block_key(row['parsed_name']), axis=1)

#print(merged_data_cleaned2)


ModuleNotFoundError: No module named 'networkx'

In [39]:
def split_and_metaphone(full_name):
    """
    Divide un nombre completo en sus partes (palabras), aplica Metaphone a cada parte,
    y las junta nuevamente con espacios.
    """
    if pd.isna(full_name) or not isinstance(full_name, str):  
        return np.nan

    parts = full_name.strip().split()

    return ' '.join(create_block_key(part) for part in parts)

# Apply the function to the 'parsed_name' column
merged_data_cleaned2['block_key'] = merged_data_cleaned2['parsed_name'].apply(split_and_metaphone)
print(merged_data_cleaned2)

   transaction_reference_id parsed_name postal_code block_key
0                         1    john doe       12345     JAN D
1                         2     jon doe       12345     JAN D
2                         3    jane doe       67890     JAN D
3                         4   jonny doe       67891    JANY D
4                         5       sonya       67892       SAN


### 1.2 Phone pre-processing

In [40]:
# Phone number normalization

combined_data['party_phone'] = combined_data['party_phone'].str.split('x').str[0]
combined_data['party_phone'] = combined_data['party_phone'].replace(r'\D', '', regex=True)
combined_data['party_phone'] = combined_data['party_phone'].str.lstrip('0')

data = combined_data[['party_phone']]
print(data)

          party_phone
0                 NaN
1        419477655328
2                 NaN
3       4119006262567
4                 NaN
...               ...
11059  13445497715686
11060   4116282024224
11061             NaN
11062             NaN
11063  51473539600789

[11064 rows x 1 columns]


In [41]:


# Paso 3: Construcción del grafo
graph = nx.Graph()

# Agregar nodos
for idx, row in data.iterrows():
    node_id = row['transaction_reference_id']
    graph.add_node(node_id, attributes=row)

# Agregar aristas basadas en similitud
for node1, node2 in possible_pairs:  # Comparar dentro de cada bloque
    sim_name = jaro_winkler_similarity(graph.nodes[node1]['attributes']['parsed_name'],
                                       graph.nodes[node2]['attributes']['parsed_name'])
    if sim_name > 0.8:
        graph.add_edge(node1, node2)

# Paso 4: Identificar componentes conexos
clusters = list(nx.connected_components(graph))

# Paso 5: Asignar IDs de cluster
cluster_mapping = {node: cluster_id for cluster_id, cluster in enumerate(clusters) for node in cluster}
data['external_id'] = data['transaction_reference_id'].map(cluster_mapping)


KeyError: 'transaction_reference_id'